In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options

In [13]:
import time
import csv
import numpy as np
import pandas as pd
import requests
import re

from IPython.core.display import HTML

In [26]:
driver = sign_in('memailn@gmail.com', 'password1!', PATH)

In [7]:
PATH = "C:\Program Files (x86)\chromedriver.exe"

In [10]:
def sign_in(email, password, PATH):
#     this function automates login process on linkedin, 
#     provide email and password as strings
    driver = webdriver.Chrome(PATH)
    time.sleep(5)
    # webdriver gets page
    driver.get("https://www.linkedin.com/")
    # pause for page to load
    time.sleep(3)
    # locate and send login email and password
    driver.find_element_by_id("session_key").send_keys(email)
    driver.find_element_by_id("session_password").send_keys(password)
    driver.find_element_by_class_name("sign-in-form__submit-button").click()
    return driver

In [49]:
def people_search(search_term, num_pages):

#                 function to automate search bar, search focus
#                 from your linkedin homepage, collecting data from
#                 search including name, location(secondary_deets), 
#                 headline(primary_deets) from condensed profiles returned from
#                 search results. requires string entry for search term
#                 and int input for number of pages (num_pages) 
#                 that you wish to scrape from results.

    # activate search bar cursor with click
    driver.find_element_by_id("global-nav-search").click()
    # send keyboard entry for search terms
    driver.find_element_by_css_selector("input.search-global-typeahead__input").send_keys(search_term)
    # send enter key to activate search
    driver.find_element_by_css_selector("input.search-global-typeahead__input").send_keys(Keys.RETURN)
    # wait for results to load
    driver.implicitly_wait(10)
    # scroll to bottom of dynamic webpage allowing load
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    # locate banner under search type results
    button = driver.find_element_by_link_text(f"See all people results")
    # js function to click banner/button to see additional results under jobs, people, or posts                      
    driver.execute_script('arguments[0].click();',button)
    # pause to allow page to load
    driver.implicitly_wait(6)
                          
    # starting with pagination page 1                       
    page_number = 1
    # get current url for page
    current_page_url = driver.current_url

    # set while loop for pagination and data collection
    while page_number < num_pages:
        print("Processing page: " + str(page_number))
    # find all results
        links = driver.find_elements_by_css_selector("span.entity-result__title a.app-aware-link")
   
        time.sleep(2)
    # iterate through results
        for link in links:
    # retrieve full name
            name = str(link.get_attribute("innerText")).split('\n')
    # add to names list
            names.append(name[0])
    # retrieve profile url
            profile_path = (str(link.get_attribute("pathname")))
    # add to urls list
            urls.append(profile_path)
    # find all primary and secondary details 
            primary_deets = driver.find_element_by_css_selector("div.entity-result__primary-subtitle.t-14.t-black")
            secondary_deets = driver.find_element_by_css_selector('div.entity-result__secondary-subtitle.t-14')
    # retrive text from primary, headline tends to be current occupation, depends on user  
            desc = primary_deets.get_attribute("innerText")
            p_details.append(desc)
    # retrive text from primary, tends to be geographical location
            loc = secondary_deets.get_attribute("innerText")
            s_details.append(loc)
    # use page_number to enter next page url 
        page_number+=1
        next_page_url = current_page_url + "&page=" + str(page_number)
        print("attempting to navigate to: " + next_page_url)
    # tell webdriver to get the next page in search results pages sequence 
        driver.get(next_page_url) 
        driver.implicitly_wait(5)
    return names, urls, p_details, s_details


In [16]:
def get_full_name():
    profile_header = driver.find_element_by_css_selector("ul.pv-top-card--list")
    # find name element from profile header portion
    header_details = profile_header.find_elements_by_tag_name("li")
    profile_name = header_details[0].get_attribute('innerText')
    return profile_name

def get_first_name():
    full_name = get_name()
    name = full_name.split(' ', 1)
    first_name = name[0]
    return first_name



In [17]:
def go_to_element(wait_element_css, select_element):
    w = WebDriverWait(driver, 10)
    # choose element that will populate without scroll, otherwise, use scroll to position
    w.until(EC.presence_of_element_located((By.CSS_SELECTOR, wait_element_css)))
    time.sleep(5)
    # background container
    element = driver.find_element_by_css_selector(select_element)
    #print("here" + str(background))
    driver.execute_script("arguments[0].scrollIntoView();", element)
    return element

In [18]:
def get_job_title(container):
    try:
        position = container.find_element_by_tag_name(
            'h3').get_attribute('innerText')
    except NoSuchElementException:
        position = 'nan'
    return position

def get_company(container):
    try:
        company = container.find_element_by_tag_name(
            'p.pv-entity__secondary-title').get_attribute('innerText')
        
    except NoSuchElementException:
        company = 'nan'
    return company

def get_work_location(container):
    try:
        location = container.find_element_by_css_selector(
            'h4.pv-entity__location').get_attribute('innerText')
        location = re.sub(r'[^\w]', ' ', location)
        location = re.sub(r'Location ', ' ', location)
        location = location.str.strip()
    except NoSuchElementException:
        location = 'nan'
    return location   

def get_dates_employed(container):
    try:
        dates = container.find_element_by_css_selector(
            "h4.pv-entity__date-range").get_attribute('innerText').split(' ', 2)[-1]

    except NoSuchElementException:
        dates = 'nan'
    return dates


In [39]:
def get_employment(job_history_digit):
#     argument input must be integer, gets single job info at a time, 
#     for example, want their most recent job, job_history_digit = 1, 
#     want the job before most recent, or second in list of jobs, job_history_digit = 2, etc
    j = int(job_history_digit) - 1
    background = go_to_element("div.profile-detail", "div[id='oc-background-section']" )
    time.sleep(4)
    # locate experience section
    try:
        exp = background.find_element_by_css_selector(
            "section[id='experience-section']")
    
    except NoSuchElementException:
        pos = 'nan'
        company = 'nan'
        location = 'nan'
        dates = 'nan'
    else:
        history = exp.find_elements_by_css_selector(
                "a[data-control-name='background_details_company']")
        details = history[j]
        pos = get_job_title(details)
        company = get_company(details)
        location = get_work_location(details)
        dates = get_dates_employed(details)

    time.sleep(2)
  


In [20]:
def get_school(school_history_digit):
    # locate education section, this works following experience section, 
    # otherwise, must locate background element see go_to_element script above 
    s = int(school_history_digit) -1
    try:
        education = background.find_element_by_css_selector(
            "section[id='education-section']")

    except NoSuchElementException:
        school_name =  'nan'
        
    else:
        schools = education.find_elements_by_css_selector(
            "a[data-control-name='background_details_school']")
        school = schools[s]
        school_name = container.find_element_by_tag_name(
        'h3').get_attribute('innerText')
# scroll to bottom to prevent automation detection flag
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    return school_name

In [21]:
def get_email():
    # scroll to top of profile where email is located
    driver.execute_script("window.scrollTo(0, 0);")
    contact_info = driver.find_element_by_css_selector(
        'a[data-control-name="contact_see_more"]')
    contact_info.click()
    time.sleep(3)
    try:
        container =  driver.find_elements_by_css_selector(
            'div.pv-contact-info__ci-container')
        email =  container[1].get_attribute('innerText')
    
    except NoSuchElementException:
        email = 'nan'
    close_popup = driver.find_element_by_css_selector(
            'button[aria-label="Dismiss" ]')
    close_popup.click()
    return email

In [22]:
def profile_connect():
    # scroll to top of profile to ensure elements can be found by webdriver
    driver.execute_script("window.scrollTo(0, 0);")
    # obtain name to personalize message
    f_name = get_first_name()
    time.sleep(2)
    # locate & click 'Connect' button located at top of profile
    driver.find_element_by_class_name('pv-s-profile-actions--connect').click()
    action_bar = driver.find_element_by_css_selector("div.artdeco-modal__actionbar")
    # locate 'Add a note' button by class
    action_bar.find_element_by_class_name('mr1').click()
    # create message using information scraped from profile
    message = f'''Hi {f_name}, I am a data scientist in the DC area. 
    I wanted to reach out, say hi, and connect with you'''.replace('\n',' ')
    time.sleep(3)
    # action to send text from message to input box
    message_input = driver.find_element_by_id('custom-message').send_keys(message)
    # send the message & connection request
    driver.find_element_by_class_name('ml1').click()

In [ ]:
names = []
urls = []
p_details = []
s_details = []
name_list = []
email_list = []
position_list = []
company_list = []
location_list = []
dates_list = []
position_list2 = []
company_list2 = []
location_list2 = []
dates_list2 = []
school_list = []
school_list2 = []

In [50]:
names = []
urls = []
p_details = []
s_details = []
people_search('artificial intelligence', 2)

Processing page: 1
attempting to navigate to: https://www.linkedin.com/search/results/all/?keywords=artificial%20intelligence&origin=GLOBAL_SEARCH_HEADER&page=2


(['Simone Vincenzi', 'Faith B.', 'Ari Chanen, PhD', 'Xiaolong M.'],
 ['/in/svincenzi',
  '/in/drfaithbradley',
  '/in/ari-chanen-phd',
  '/in/xiaolongmiao'],
 ['AI and Machine Learning at Ericsson Global Artificial Intelligence Accelerator',
  'AI and Machine Learning at Ericsson Global Artificial Intelligence Accelerator',
  'AI and Machine Learning at Ericsson Global Artificial Intelligence Accelerator',
  'AI and Machine Learning at Ericsson Global Artificial Intelligence Accelerator'],
 ['Santa Cruz, CA', 'Santa Cruz, CA', 'Santa Cruz, CA', 'Santa Cruz, CA'])

In [ ]:
driver.forward()
driver.back()

In [ ]:
driver.quit()

In [48]:
names

[]

In [38]:
names

[]